**Importing necessary libraries**

In [1]:
import pandas as pd
import numpy as np
import os
import cv2

from keras.models import load_model

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

**Loading model and cascade classifier to detect face**

In [2]:
model=load_model('best-cnn-aug')

In [3]:
face_clsfr=cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')

**Reading video , detecting face and preprocessing it accordingly to predict till escape button is given as input**

In [4]:
source=cv2.VideoCapture(0)

labels_dict={0:'NO MASK',1:'MASK'}
color_dict={0:(0,0,255),1:(0,255,0)}

while(True):

    ret,img=source.read()
    image=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(image,1.3,5)  

    for x,y,w,h in faces:
    
        face_img=image[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(64,64))        
        normalized=resized/255
        reshaped=np.reshape(normalized,(1,64,64,1))
        
        result=model.predict(reshaped)
        y_hat=np.argmax(result,axis=1)[0]               
        
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[y_hat],2)
        cv2.rectangle(img,(x,y),(x+w,y),color_dict[y_hat],-1)
        cv2.putText(img, labels_dict[y_hat], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)  
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()